In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot
import sklearn.linear_model
import matplotlib.pyplot as plt
import pyreadstat
import datetime
import FG_Methods
import tabulate




In [2]:
df_path = "C:\\Users\\fabio\\OneDrive\\Documents\\Studies\\Social_Research\\data\\ZA7780_v1-0-0.dta"
df = pd.read_stata(df_path, convert_categoricals=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38718 entries, 0 to 38717
Columns: 479 entries, studyno1 to w87
dtypes: float64(284), int16(5), int32(3), int8(184), object(3)
memory usage: 92.7+ MB


In [ ]:
#Output Dataframe as csv

project_folder = "C:\\Users\\fabio\\OneDrive\\Documents\\Studies\\Social_Research\\"

df_column_headers = ""
for i in df.columns:
    if i == df.columns[0]:
        df_column_headers = i
    else:
        df_column_headers = df_column_headers + ", " + i

np.savetxt(project_folder + 'social_research_dataframe.csv',  np.asarray(df), header=df_column_headers, delimiter=',', fmt='%s')


In [ ]:
print("Hello")

In [20]:
"""Note this is currently unweighted"""
"""This is the old version of the data, since I decide to go for a dictionary approach"""

def fg_counter(counter_value, total_iterations_qty, start, number_of_updates_required_for_total_run = 10, update_counter = False):
    if float(counter_value) % int(total_iterations_qty / number_of_updates_required_for_total_run) == 0:
        print("-----")
        print(datetime.now())
        PC = float(float(counter_value) / total_iterations_qty)
        print(str(counter_value) + " / " + str(total_iterations_qty))
        print(PC)
        print("end estimate @: " + str((datetime.now() - start) * (total_iterations_qty / counter_value) + datetime.now()))
        if update_counter == True:
            return PC    

def transfer_columns(new_df, original_df, target_string, target_string_at_start=True, data_columns_exact_match=False):
    #this method scans every column header of original_df and if target_string is contained within a header, at column is added to new df.
    #this allows me to iterativelly add groups of columns from the original datarame to the reduced datafrom for the analysis
    #target string must be entered as a string
    columns = original_df.columns
    if type(target_string) is str:
        target_string = [target_string]
    
    for string_ in target_string:
        for col in columns:
            if col.find(string_) >= 0 and target_string_at_start==False: 
                new_df = pd.concat([new_df, original_df[col]], axis=1)
            if col.find(string_) == 0 and target_string_at_start==True:
                new_df = pd.concat([new_df, original_df[col]], axis=1)
        
    return new_df

def return_stats_on_group(original_df, group_column, group_identifiers, data_group, data_columns_exact_match=False):

    group_flag = []
    group_flag = np.array(original_df[group_column] == group_identifiers[0])

    for i in range(1, len(group_identifiers)):
        group_flag_ = np.array(original_df[group_column] == i)
        group_flag = group_flag or group_flag_
    results_df = original_df[group_flag]
    results_df = results_df.drop(group_column, axis=1) 

    #print(type(float("{:.2f}".format(results_df.values.mean()))))

    return float("{:.2f}".format(results_df.values.mean())), float("{:.2f}".format(results_df.values.std()))

#create data tables
df_results = pd.DataFrame()
df_reduced = pd.DataFrame()
df_reduced = transfer_columns(df_reduced, df, ["isocntry", "qa1a_"], target_string_at_start=True, data_columns_exact_match=False)    

#return results
data_group_name = "qa1a"
unit_group_name = "GB"
output_mean, output_std = return_stats_on_group(df_reduced, "isocntry", ["GB"], ["qa1a_"], data_columns_exact_match=False)

#enter data to series
result_dict = {(data_group_name, "mean"): output_mean, (data_group_name, "std"): output_std}
result_series = pd.Series(result_dict, name=unit_group_name)
df_results = df_results.append(result_series)

df_results

#I need to reform the data as a multi-index dictionary.
#With the key being: Country, Value Name, Value


C:\Users\fabio\AppData\Local\Temp\ipykernel_9148\3966432523.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(result_series)


qa1a      
    mean   std
GB  2.62  0.83

In [ ]:
test_columns = ["qa1a_1", "qa1a_2", "qa1a_3"]
df_test = df.loc[df["isocntry"] == "GB"]

test = df_test[test_columns]

In [ ]:
test

In [ ]:
#This code is for experimenting with mutli-dimensional python dictionaries

test_dic = {
('A', 'a', "mean"): [1, 2, 3, 4, 5],
('A', 'a', "std"): [1, 2, 3, 4, 5],
('A', 'b', "mean"): [6, 7, 8, 9, 1],
('A', 'b', "std"): [6, 7, 8, 9, 1],
('B', 'a', "mean"): [2, 3, 4, 5, 6],
('B', 'a', "std"): [2, 3, 4, 5, 6],
('B', 'b', "mean"): [7, 8, 9, 1, 2],
('B', 'b', "std"): [7, 8, 9, 1, 2] }



test_dic_insert = {
('a', "mean"): [1, 2, 3, 4, 5],
('a', "std"): [1, 2, 3, 4, 5],
('b', "mean"): [6, 7, 8, 9, 1],
('b', "std"): [6, 7, 8, 9, 1],
}


#def update_key(dict):
#    for entry in dict:

dict_1 = {("group_1", "mean"): 1, ("group_1", "std"): 2,
          ("group_2", "mean"): 2, ("group_2", "std"): 0.5}
test_series_1 = pd.Series(dict_1, name="GB")

dict_2 = {("group_1", "mean"): 3, ("group_1", "std"): 3,
          ("group_2", "mean"): 5, ("group_2", "std"): 8}
test_series_2 = pd.Series(dict_2, name="IT")

pd.DataFrame([test_series_1, test_series_2])


